In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb  # Adicione esta linha para importar XGBoost
from sklearn.preprocessing import LabelEncoder

# Carregar o dataset
base_hotel = pd.read_csv(r'/content/Hotel Reservations.csv')

# Classificação de preço
def classify_price(price):
    if price <= 85:
        return 1  # Mantendo o formato 1, 2, 3
    elif 85 < price < 115:
        return 2  # Mantendo o formato 1, 2, 3
    else:
        return 3  # Mantendo o formato 1, 2, 3

# Criar a coluna de label 'label_avg_price_per_room'
base_hotel['label_avg_price_per_room'] = base_hotel['avg_price_per_room'].apply(classify_price)
base_hotel = base_hotel.drop(columns=['avg_price_per_room'])

# Reorganizar as colunas
colunas = []
colunas.append('label_avg_price_per_room')
for i in range(len(base_hotel.columns[:-1])):
    print(base_hotel.columns[i])
    colunas.append(base_hotel.columns[i])

base_hotel = base_hotel[colunas]

# Lista de colunas a serem removidas
colunas_para_remover = ['Booking_ID', 'arrival_date', 'market_segment_type',
                         'repeated_guest', 'no_of_previous_cancellations',
                         'no_of_previous_bookings_not_canceled', 'booking_status']

# Filtrar e remover as colunas existentes
colunas_existentes = [col for col in colunas_para_remover if col in base_hotel.columns]
base_hotel = base_hotel.drop(columns=colunas_existentes, axis=1).copy()

# Aplicando One-Hot Encoding
base_hotel = pd.get_dummies(base_hotel, prefix=['type_of_meal_plan', 'required_car_parking_space',
                                                'room_type_reserved', 'arrival_year', 'arrival_month',
                                                'no_of_special_requests'],
                             columns=['type_of_meal_plan', 'required_car_parking_space',
                                      'room_type_reserved', 'arrival_year', 'arrival_month',
                                      'no_of_special_requests'])

# Definindo X e y
X = base_hotel.iloc[:, 1:].values
X = np.array(X).astype('float32')
y = base_hotel.iloc[:, 0].values

# Codificar os rótulos para começarem de 0
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir o conjunto de dados em treinamento e teste
X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(X, y_encoded, test_size=0.3, random_state=0)

In [ ]:
# Extrair as features (X) e o target (Y)
X = base_hotel.iloc[:, 1:].values  # Seleciona todas as colunas exceto a primeira para X
Y = base_hotel.iloc[:, 0].values   # Seleciona a primeira coluna (label_avg_price_per_room) para Y

# Codificação das variáveis categóricas
label_encoders = {}
for i, column in enumerate(base_hotel.columns[1:], start=1):
    if base_hotel[column].dtype == 'object':
        le = LabelEncoder()
        X[:, i-1] = le.fit_transform(X[:, i-1])
        label_encoders[column] = le

# Converte X para float32
X = np.array(X).astype('float32')

# Codifique os rótulos para inteiros consecutivos, começando de 0
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

# Dividir o conjunto de dados em treinamento e teste
X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(X, Y_encoded, test_size=0.3, random_state=0)

# Configuração dos hiperparâmetros para RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 200],  # Número de árvores
    'max_depth': [10, 15],  # Profundidade máxima da árvore
    'learning_rate': [0.05, 0.1],  # Taxa de aprendizado
    'subsample': [0.9, 1.0],  # Proporção de amostras usadas para ajustar cada árvore
    'colsample_bytree': [0.8, 0.9],  # Proporção de colunas usadas para ajustar cada árvore
    'gamma': [0, 0.2],  # Regularização por complexidade de modelo
    'min_child_weight': [1, 3],  # Peso mínimo da folha
    'reg_alpha': [0, 0.1],  # Regularização L1
    'reg_lambda': [1, 1.5]  # Regularização L2
}

# Inicializar o classificador XGBoost
model_xgb = xgb.XGBClassifier(
    eval_metric='mlogloss',  # Métrica para problemas de classificação multiclasse
    num_class=len(np.unique(Y_encoded))  # Número de classes
)

# Inicializar o RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model_xgb,
    param_distributions=param_distributions,
    n_iter=100,  # Número de combinações aleatórias a serem testadas
    scoring='accuracy',
    cv=3,  # Número de folds para cross-validation
    n_jobs=-1,  # Utiliza todos os núcleos do processador
    verbose=2,  # Mostra detalhes sobre o progresso
    random_state=0  # Para reprodutibilidade
)

# Treinar o RandomizedSearchCV
random_search.fit(X_treinamento, Y_treinamento)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros encontrados:\n", random_search.best_params_)

# Melhor modelo encontrado
best_model = random_search.best_estimator_

# Fazer previsões
Y_pred_xgb_encoded = best_model.predict(X_teste)

# Decodificar as previsões de volta para os rótulos originais
Y_pred_xgb = label_encoder.inverse_transform(Y_pred_xgb_encoded)

# Decodificar os rótulos de teste para os rótulos originais
Y_teste_decoded = label_encoder.inverse_transform(Y_teste)

# Avaliar o modelo
print("XGBoost Accuracy Score:", accuracy_score(Y_teste_decoded, Y_pred_xgb))
print("XGBoost Classification Report:\n", classification_report(Y_teste_decoded, Y_pred_xgb))

# Verifique as classes únicas no conjunto de teste e previsões
print("Classes únicas no conjunto de teste:", np.unique(Y_teste_decoded))
print("Classes únicas nas previsões:", np.unique(Y_pred_xgb))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb  # Adicione esta linha para importar XGBoost
from sklearn.preprocessing import LabelEncoder

# Carregar o dataset
base_hotel = pd.read_csv(r'/content/Hotel Reservations.csv')

# Classificação de preço
def classify_price(price):
    if price <= 85:
        return 1  # Mantendo o formato 1, 2, 3
    elif 85 < price < 115:
        return 2  # Mantendo o formato 1, 2, 3
    else:
        return 3  # Mantendo o formato 1, 2, 3

# Criar a coluna de label 'label_avg_price_per_room'
base_hotel['label_avg_price_per_room'] = base_hotel['avg_price_per_room'].apply(classify_price)
base_hotel = base_hotel.drop(columns=['avg_price_per_room'])

# Reorganizar as colunas
colunas = []
colunas.append('label_avg_price_per_room')
for i in range(len(base_hotel.columns[:-1])):
    colunas.append(base_hotel.columns[i])

base_hotel = base_hotel[colunas]

# Lista de colunas a serem removidas
colunas_para_remover = ['Booking_ID', 'arrival_date', 'market_segment_type',
                         'repeated_guest', 'no_of_previous_cancellations',
                         'no_of_previous_bookings_not_canceled', 'booking_status']

# Filtrar e remover as colunas existentes
colunas_existentes = [col for col in colunas_para_remover if col in base_hotel.columns]
base_hotel = base_hotel.drop(columns=colunas_existentes, axis=1).copy()

# Aplicando One-Hot Encoding
base_hotel = pd.get_dummies(base_hotel, prefix=['type_of_meal_plan', 'required_car_parking_space',
                                                'room_type_reserved', 'arrival_year', 'arrival_month',
                                                'no_of_special_requests'],
                             columns=['type_of_meal_plan', 'required_car_parking_space',
                                      'room_type_reserved', 'arrival_year', 'arrival_month',
                                      'no_of_special_requests'])

# Definindo X e y
X = base_hotel.iloc[:, 1:].values
X = np.array(X).astype('float32')
y = base_hotel.iloc[:, 0].values

# Codificar os rótulos para começarem de 0
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir o conjunto de dados em treinamento e teste
X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(X, y_encoded, test_size=0.3, random_state=0)

# Configuração dos hiperparâmetros para RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 200],  # Número de árvores
    'max_depth': [10, 15],  # Profundidade máxima da árvore
    'learning_rate': [0.05, 0.1],  # Taxa de aprendizado
    'subsample': [0.9, 1.0],  # Proporção de amostras usadas para ajustar cada árvore
    'colsample_bytree': [0.8, 0.9],  # Proporção de colunas usadas para ajustar cada árvore
    'gamma': [0, 0.2],  # Regularização por complexidade de modelo
    'min_child_weight': [1, 3],  # Peso mínimo da folha
    'reg_alpha': [0, 0.1],  # Regularização L1
    'reg_lambda': [1, 1.5]  # Regularização L2
}

# Inicializar o classificador XGBoost
model_xgb = xgb.XGBClassifier(
    eval_metric='mlogloss',  # Métrica para problemas de classificação multiclasse
    num_class=len(np.unique(y_encoded))  # Número de classes
)

# Inicializar o RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model_xgb,
    param_distributions=param_distributions,
    n_iter=100,  # Número de combinações aleatórias a serem testadas
    scoring='accuracy',
    cv=3,  # Número de folds para cross-validation
    n_jobs=-1,  # Utiliza todos os núcleos do processador
    verbose=2,  # Mostra detalhes sobre o progresso
    random_state=0  # Para reprodutibilidade
)

# Treinar o RandomizedSearchCV
random_search.fit(X_treinamento, Y_treinamento)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros encontrados:\n", random_search.best_params_)

# Melhor modelo encontrado
best_model = random_search.best_estimator_

# Fazer previsões
Y_pred_xgb_encoded = best_model.predict(X_teste)

# Decodificar as previsões de volta para os rótulos originais
Y_pred_xgb = label_encoder.inverse_transform(Y_pred_xgb_encoded)

# Decodificar os rótulos de teste para os rótulos originais
Y_teste_decoded = label_encoder.inverse_transform(Y_teste)

# Avaliar o modelo
print("XGBoost Accuracy Score:", accuracy_score(Y_teste_decoded, Y_pred_xgb))
print("XGBoost Classification Report:\n", classification_report(Y_teste_decoded, Y_pred_xgb))

# Verifique as classes únicas no conjunto de teste e previsões
print("Classes únicas no conjunto de teste:", np.unique(Y_teste_decoded))
print("Classes únicas nas previsões:", np.unique(Y_pred_xgb))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# Carregar o dataset
base_hotel = pd.read_csv(r'/content/Hotel Reservations.csv')

# Função para classificar o preço
def classify_price(price):
    if price <= 85:
        return 1

    elif price > 85 and price < 115:
        return 2
    else:
        return 3

# Criar a coluna de label 'label_avg_price_per_room'
base_hotel['label_avg_price_per_room'] = base_hotel['avg_price_per_room'].apply(classify_price)
base_hotel = base_hotel.drop(columns=['avg_price_per_room'])

# Reorganizar as colunas
colunas = ['label_avg_price_per_room'] + [col for col in base_hotel.columns if col != 'label_avg_price_per_room']
base_hotel = base_hotel[colunas]

# Lista de colunas a serem removidas
colunas_para_remover = ['Booking_ID', 'arrival_date', 'market_segment_type',
                         'repeated_guest', 'no_of_previous_cancellations',
                         'no_of_previous_bookings_not_canceled', 'booking_status']

# Filtrar e remover as colunas existentes
colunas_existentes = [col for col in colunas_para_remover if col in base_hotel.columns]
base_hotel = base_hotel.drop(columns=colunas_existentes, axis=1).copy()

# Aplicando One-Hot Encoding
base_hotel = pd.get_dummies(base_hotel, prefix=['type_of_meal_plan', 'required_car_parking_space',
                                                'room_type_reserved', 'arrival_year', 'arrival_month',
                                                'no_of_special_requests'],
                             columns=['type_of_meal_plan', 'required_car_parking_space',
                                      'room_type_reserved', 'arrival_year', 'arrival_month',
                                      'no_of_special_requests'])

# Definindo X e y
X = base_hotel.drop(columns=['label_avg_price_per_room']).values.astype('float32')
y = base_hotel['label_avg_price_per_room'].values

# Codificar os rótulos para começarem de 0
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir o conjunto de dados em treinamento e teste
X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(X, y_encoded, test_size=0.3, random_state=0)

# Configuração dos hiperparâmetros para RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 15],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.9, 1.0],
    'colsample_bytree': [0.8, 0.9],
    'gamma': [0, 0.2],
    'min_child_weight': [1, 3],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [1, 1.5]
}

# Inicializar o classificador XGBoost
model_xgb = xgb.XGBClassifier(
    eval_metric='mlogloss',
    num_class=len(np.unique(y_encoded))
)

# Inicializar o RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model_xgb,
    param_distributions=param_distributions,
    n_iter=20,  # Reduzindo o número de combinações para acelerar
    scoring='accuracy',
    cv=3,
    n_jobs=-1,
    verbose=2,
    random_state=0
)

# Treinar o RandomizedSearchCV
random_search.fit(X_treinamento, Y_treinamento)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros encontrados:\n", random_search.best_params_)

# Melhor modelo encontrado
best_model = random_search.best_estimator_

# Fazer previsões
Y_pred_xgb_encoded = best_model.predict(X_teste)

# Decodificar as previsões de volta para os rótulos originais
Y_pred_xgb = label_encoder.inverse_transform(Y_pred_xgb_encoded)

# Decodificar os rótulos de teste para os rótulos originais
Y_teste_decoded = label_encoder.inverse_transform(Y_teste)

# Avaliar o modelo
print("XGBoost Accuracy Score:", accuracy_score(Y_teste_decoded, Y_pred_xgb))
print("XGBoost Classification Report:\n", classification_report(Y_teste_decoded, Y_pred_xgb))

# Verifique as classes únicas no conjunto de teste e previsões
print("Classes únicas no conjunto de teste:", np.unique(Y_teste_decoded))
print("Classes únicas nas previsões:", np.unique(Y_pred_xgb))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Melhores hiperparâmetros encontrados:
 {'subsample': 0.9, 'reg_lambda': 1.5, 'reg_alpha': 0, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 15, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 0.9}
XGBoost Accuracy Score: 0.7953689240099238
XGBoost Classification Report:
               precision    recall  f1-score   support

           1       0.80      0.79      0.80      3258
           2       0.76      0.76      0.76      4006
           3       0.83      0.84      0.83      3619

    accuracy                           0.80     10883
   macro avg       0.80      0.80      0.80     10883
weighted avg       0.80      0.80      0.80     10883

Classes únicas no conjunto de teste: [1 2 3]
Classes únicas nas previsões: [1 2 3]
